# Hyperparameters

Hyperparameters are tunable in training a neural network, you can change the hyperprameters to find the best performance.

In nerual networks, there are several hyperparameters that you can tune, like: Number of Hidden Layers; Number of Neurons per Hidden Layers; Learning Rate; Batch Size and Optimizer. 
For example, you can change the value of Learning rate to see what your training will be.
Further information about Hyperparameter Tuning can refer to: https://www.tech-quantum.com/basic-hyperparameter-tuning-for-neural-networks/.

In this section, we just tune one parameter once. You can tune several parameters from a parameter search space according to [websit](https://machinelearningmastery.com/how-to-grid-search-hyperparameters-for-pytorch-models/) and [ray tune](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html).



In [1]:
#@title Import Dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
#@title Define Hyperparameters

# set img_size = (28,28) ---> 28*28=784 pixels in total
input_size = 784 
# number of nodes at hidden layer
hidden_size = 500
# number of output classes discrete range [0,9]
num_classes = 10 
# number of times which the entire dataset is passed throughout the model
num_epochs = 30 
# the size of input data took for one iteration
batch_size = 100 
# learning rate 
lr = 1e-3  

In [3]:
#@title Downloading MNIST data

train_data = dsets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

test_data = dsets.MNIST(root = './data', train = False,
                       transform = transforms.ToTensor())

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.16MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 418kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 878kB/s] 
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.82MB/s]


In [4]:
#@title Loading the data

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

In [5]:
#@title Define model class

class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()  # Relu activation function, you can also use others like Tanh, Sigmold, etc.
    self.fc2 = nn.Linear(hidden_size, num_classes)
  
  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [6]:
#@title Build the model

net = Net(input_size, hidden_size, num_classes)
if torch.cuda.is_available():
  net.cuda()

In [7]:
#@title Define loss-function & optimizer

loss_function = nn.CrossEntropyLoss()

# Adam optimizer, you can also use AdaGrad or RMSProp, etc.
optimizer = torch.optim.Adam( net.parameters(), lr=lr) 

In [8]:
#@title Training the model

for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    # if you have GPU, you can set as  .cuda()
    # images = Variable(images.view(-1,28*28)).cuda()
    images = Variable(images.view(-1,28*28))
    # if you have GPU, you can set as  .cuda()
    # labels = Variable(labels).cuda()
    labels = Variable(labels)
    
    optimizer.zero_grad()
    outputs = net(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.item()))


Epoch [1/30], Step [100/600], Loss: 0.2215
Epoch [1/30], Step [200/600], Loss: 0.3249
Epoch [1/30], Step [300/600], Loss: 0.4051
Epoch [1/30], Step [400/600], Loss: 0.1336
Epoch [1/30], Step [500/600], Loss: 0.1974
Epoch [1/30], Step [600/600], Loss: 0.2126
Epoch [2/30], Step [100/600], Loss: 0.1028
Epoch [2/30], Step [200/600], Loss: 0.0600
Epoch [2/30], Step [300/600], Loss: 0.0892
Epoch [2/30], Step [400/600], Loss: 0.0945
Epoch [2/30], Step [500/600], Loss: 0.1585
Epoch [2/30], Step [600/600], Loss: 0.1495
Epoch [3/30], Step [100/600], Loss: 0.1415
Epoch [3/30], Step [200/600], Loss: 0.0665
Epoch [3/30], Step [300/600], Loss: 0.1437
Epoch [3/30], Step [400/600], Loss: 0.0834
Epoch [3/30], Step [500/600], Loss: 0.0802
Epoch [3/30], Step [600/600], Loss: 0.0799
Epoch [4/30], Step [100/600], Loss: 0.0803
Epoch [4/30], Step [200/600], Loss: 0.0757
Epoch [4/30], Step [300/600], Loss: 0.0473
Epoch [4/30], Step [400/600], Loss: 0.0208
Epoch [4/30], Step [500/600], Loss: 0.1034
Epoch [4/30

In [9]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for images,labels in test_gen:
  # if you have GPU, you can set as  .cuda()
  # images = Variable(images.view(-1,28*28)).cuda()
  images = Variable(images.view(-1,28*28))
  # abels = labels.cuda()
  labels = labels
  
  output = net(images)
  _, predicted = torch.max(output,1)
  correct += (predicted == labels).sum()
  total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

Accuracy of the model: 98.340 %


When **num_epochs** = 20, the result is up to 98.190%. When we change the **num_epochs**, we can obatain other results. We can iterate the parameters and plot the best results.